In [1]:
import sys
import os

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))

In [2]:
from pathlib import Path
from src.utils.config_loader import load_config

base_dir = Path(os.getcwd()).parent

config = load_config(base_dir / 'secrets.yaml')

In [3]:
import json

with open(base_dir / 'data/val_set.json', 'r') as f:
    data = json.load(f)

In [4]:
def format_examples(example_subset): 
    # Formats the examples into a string for later prompt
    formatted = []
    for i, ex in enumerate(example_subset):
        formatted.append(
            f"Eksempel {i+1}:\n"
            f"Versjon 1:\n{ex['version_1']}\n\n"
            f"Versjon 2:\n{ex['version_2']}\n\n"
            f"Oppsummering:\n{ex['ref_summary']}\n##\n"
        )
    
    return "\n".join(formatted)

ids = [27]
#ids = [27, 39, 23]

examples = [next(ex for ex in data if ex["id"] == id) for id in ids]

formatted_examples = format_examples(examples)

print(formatted_examples)

Eksempel 1:
Versjon 1:
Det tillates maks en boenhet per tomt.

Versjon 2:
Det tillates inntil to boenheter per tomt.

Oppsummering:
Tillatt antall boenheter per tomt er økt fra en til to.
##



In [ ]:
import torch
import transformers

class LlamaForSummarization:
    def __init__(self, model_path: str):
        self.model_id = model_path
        self.pipeline = transformers.pipeline(
            'text-generation',
            model=self.model_id,
            model_kwargs={'torch_dtype': torch.float16} # FP16 for faster inference
        )
        tokenizer = self.pipeline.tokenizer
        tokenizer.pad_token_id = tokenizer.eos_token_id 

        self.terminators = tokenizer.eos_token_id
  
    def format_prompt(self, v1, v2):
        prompt = f"""\
    Du er en fagperson med ekspertise innen arealplanlegging og reguleringsplaner. Din oppgave er å bistå saksbehandlere med å identifisere og oppsummere forskjellene mellom to versjoner av tekst hentet fra en reguleringsplan. Dersom versjon 1 inneholder tekst, men ikke versjon 2, betyr det at teksten har blitt fjernet. Hvis det er omvendt har teksten blitt lagt til. Ikke referer til versjon 1 eller versjon 2 i oppsummeringen. Beskriv kun hva som er fjernet, lagt til eller endret.

    - Oppsummeringen skal være så kort som mulig.
    - Unngå unødvendige detaljer.
    - Svar utelukkende med oppsummeringsteksten.

    Bruk eksemplene som hjelp til å velge riktig ord.

    {formatted_examples}

    Oppsummer forskjellene mellom følgende to versjoner:

    Versjon 1:
    {v1}
    
    Versjon 2:
    {v2}
    
    Oppsummering:
    """
        return prompt

    def summarize(self, v1, v2, max_tokens=50, temperature=0.0, top_p=0.9):

        prompt = self.format_prompt(v1, v2) 

        outputs = self.pipeline(
            prompt,
            max_new_tokens=max_tokens,
            eos_token_id=self.terminators,
            pad_token_id=self.terminators,
            do_sample=False, 
            temperature=temperature, 
            top_p=top_p 
        )
        
        generated_text = outputs[0]['generated_text'][len(prompt):].strip()
       
        return generated_text

In [6]:
import json

with open(base_dir / 'data/val_set.json', 'r') as f:
    data = json.load(f)

In [ ]:
from tqdm import tqdm

model_path = 'meta-llama/Meta-Llama-3-8B-Instruct' 
model = LlamaForSummarization(model_path)

results = []

data_sample = data #data[:1]

for item in tqdm(data_sample):

    version_1 = item['version_1']
    version_2 = item['version_2']
    ref_summary = item['ref_summary'] 

    output = model.summarize(version_1, version_2)

    results.append({
            'id': item['id'],
            'ref_summary': ref_summary,
            'model_summary': output
        })

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Device set to use cuda:0
  0%|          | 0/35 [00:00<?, ?it/s]/opt/conda/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/opt/conda/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:636: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
100%|██████████| 35/35 [00:53<00:00,  1.54s/it]


In [8]:
with open(base_dir / 'results/Meta-Llama-3-8B-Instruct_NOR_ONESHOT.jsonl', 'w') as f:
    for result in results:
        json.dump(result, f, ensure_ascii=False)
        f.write('\n')